In [1]:
# Import Dependencies

import json
import numpy as np
import pandas as pd
import matplotlib as plt
import requests
import time
from config import api_key


In [ ]:
#total_pop = "B01001_001E"

## Age Group Codes
#Males age 18 to 19 = B01001_007E
#Males age 20 = B01001_008E
#Males age 21 = B01001_009E
#Males age 22 to 24 = B01001_010E
#Males age 25 to 29 = B01001_011E
#Males age 30 to 34 = B01001_012E
#Males age 35 to 39 = B01001_013E
#Males age 40 to 44 = B01001_014E

#Females age 18 to 19 = B01001_031E
#Females age 20 = B01001_032E
#Females age 21 = B01001_033E
#Females age 22 to 24 = B01001_034E
#Females age 25 to 29 = B01001_035E
#Females age 30 to 34 = B01001_036E
#Females age 35 to 39 = B01001_037E
#Females age 40 to 44 = B01001_038E

## Ethnicity Group Codes
#White = B02001_002E
#Black = B02001_003E
#Latino = B03001_003E
#Native Indian = B02001_004E
#Asian = B02001_005E
#Pacific Islander = B02001_006E
#Other = B02001_007E

## NEW Income Level
#income_pop = B25075_001E
#Less than $10,000 = B25075_002E
#$10,000 to $14,999 = B25075_003E
#$15,000 to $19,999 = B25075_004E
#$20,000 to $24,999 = B25075_005E
#$25,000 to $29,999 = B25075_006E
#$30,000 to $34,999 = B25075_007E
#$35,000 to $39,999 = B25075_008E
#$40,000 to $49,999 = B25075_009E
#$50,000 to $59,999 = B25075_010E
#$60,000 to $69,999 = B25075_011E
#$70,000 to $79,999 = B25075_012E
#$80,000 to $89,999 = B25075_013E
#$90,000 to $99,999 = B25075_014E
#$100,000 to $124,999 = B25075_015E
#$125,000 to $149,999 = B25075_016E
#$150,000 to $174,999 = B25095_017E
#$175,000 to $199,999 = B25075_018E
#$200,000 to $249,999 = B25075_019E
#$250,000 to $299,999 = B25075_020E

## Income Level
#income_pop = B25095_001E
#Less than $10,000 = B25095_002E
#$10,000 to $19,999 = B25095_011E
#$20,000 to $34,999 = B25095_020E
#$35,000 to $49,999 = B25095_029E
#$50,000 to $74,999 = B25095_038E
#$75,000 to $99,999 = B25095_047E
#$100,000 to $149,999 = B25095_056E
#$150,000 or more = B25095_065E

## Education Level
#edu_pop = B23006_001E
#Less than high school graduate = B23006_002E
#High school graduate = B23006_009E
#Some college or associates = B23006_016E
#Bachelor's or higher = B23006_023E


In [2]:
# Build query URL for exploring Census API

# Base URL
url = "https://api.census.gov/data/2016/acs/acs5?get=NAME"

# API Key call, note this should always be at the end of the URL
key = "&key=" + api_key

# Call for states, note that the State's designated number should follow the colon
state = "&in=state:"

# Call for individual counties, note that you must make the county call with the state call, it cannot be called
# by itself. Also, the county number should follow the colon
county = "&for=county:"

# Places API Call, which allows for specific cities to be referred to. Requires the "state" call to be made as well,
# otherwise it might return an error.

city = "&for=place:"

# Metropolitan API call

metro = "&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:"

# ZIP code ZCTA API call

zip_code = "&for=zip%20code%20tabulation%20area:"

# Basic query url structure should look like this (Note that the "*" is used to call all values, so all states
# and cities within that state)
query_url = url + city + "04000" + state + "13" + key

# Json request
request = requests.get(query_url).json()

# Display result of request
request


[['NAME', 'state', 'place'], ['Atlanta city, Georgia', '13', '04000']]

In [3]:
# URLs with codes built in to make appropriate calls

# Base URL

base_url = "https://api.census.gov/data/"

# Age URL
age_url = "/acs/acs5?get=NAME,B01001_001E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,"\
        +"B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,"\
        + "B01001_036E,B01001_037E,B01001_038E"

# Ethnicity URL
ethnicity_url = "/acs/acs5?get=NAME,B01001_001E,B02001_002E,B02001_003E,B03001_003E,B02001_005E"

# Income URL
income_url = "/acs/acs5?get=NAME,B25075_001E,B25075_002E,B25075_003E,B25075_004E,"\
        +"B25075_005E,B25075_006E,B25075_007E,B25075_008E,B25075_009E,B25075_010E,B25075_011E,B25075_012E,B25075_013E,"\
        +"B25075_014E,B25075_015E,B25075_016E,B25075_017E,B25075_018E,B25075_019E,B25075_020E"

# Education URL
edu_url = "/acs/acs5?get=NAME,B23006_001E,B23006_002E,B23006_009E,B23006_016E,B23006_023E"

# List of URLs for iteration in API

year_list = ["2012","2013","2014","2015","2016"]
url_list = [age_url,ethnicity_url,income_url,edu_url]


In [4]:
# Dictionary with state names as keys and the values being the corresponding code in the Census API
state_dict = {"Georgia":"13","Washington":"53","Florida":"12","New York":"36","California":"06","New Jersey":"34",\
               "Oregon":"41","Minnesota":"27","Kansas":"20","Massachusetts":"25","Utah":"49","Washington D.C.":"11",\
               "Texas":"48","Illinois":"17","Pennsylvania":"42","Ohio":"39","Colorado":"08"}

# Dictionary with city names as keys and the values being the corresponding code in the Census API
city_dict = {"Atlanta":"04000", "Seattle":"63000", "Orlando":"53000", "New York":"51000", "Carson":"11530",\
              "Harrison":"51000", "Portland":"59000", "Minneapolis":"43000", "San Jose":"68000", "Kansas City":"36000",\
              "Foxborough":"24855", "Sandy":"67440", "Washington D.C.":"50000", "Houston":"35000",\
              "Bridgeview":"08225", "Chester":"13208", "Columbus":"18000", "Commerce City":"16495", "Frisco":"27684"}

# List containing each state code in corresponding order for use in API call iteration
state_codes = ["13","53","12","36","06","34","41","27","06","20","25","49","11","48","17","42","39","08","48"]

# List containing each city code in corresponding order for use in API call iteration
city_codes = ["04000","63000","53000","51000","11530","51000","59000","43000","68000","36000","24855","67440","50000",\
             "35000","08225","13208","18000","16495","27684"]


In [5]:
# API request iteration format (NOTE: running this cell takes time due to the time.sleep! Only run
# if you want the data itself! Running this will take at least 8 MINUTES before it finishes)

# Initialize lists to store data retrieved from the API

age_list = []
ethnicity_list = []
income_list = []
edu_list = []

counter = 1

for year in year_list:

    for num in range(19):
    
        for url in url_list:
    
            # Query URL construction that pulls from the city & state codes lists
            query_url = base_url + year + url + city + city_codes[num] + state + state_codes[num] + key
        
            try:
                request = requests.get(query_url).json()
        
                if url == age_url:
                    age_list.append(request[1])
                elif url == ethnicity_url:
                    ethnicity_list.append(request[1])
                elif url == income_url:
                    income_list.append(request[1])
                elif url == edu_url:
                    edu_list.append(request[1])
            
                time.sleep(1)
        
            except Exception:
                print(f"There was an error with this url: {query_url}")
    
        print(f"#{counter} for Year {year} has completed all calls")
    
        counter += 1
        
    counter = 1


#1 for Year 2012 has completed all calls
#2 for Year 2012 has completed all calls
#3 for Year 2012 has completed all calls
#4 for Year 2012 has completed all calls
#5 for Year 2012 has completed all calls
#6 for Year 2012 has completed all calls
#7 for Year 2012 has completed all calls
#8 for Year 2012 has completed all calls
#9 for Year 2012 has completed all calls
#10 for Year 2012 has completed all calls
#11 for Year 2012 has completed all calls
#12 for Year 2012 has completed all calls
#13 for Year 2012 has completed all calls
#14 for Year 2012 has completed all calls
#15 for Year 2012 has completed all calls
#16 for Year 2012 has completed all calls
#17 for Year 2012 has completed all calls
#18 for Year 2012 has completed all calls
#19 for Year 2012 has completed all calls
#1 for Year 2013 has completed all calls
#2 for Year 2013 has completed all calls
#3 for Year 2013 has completed all calls
#4 for Year 2013 has completed all calls
#5 for Year 2013 has completed all calls
#6 for

In [6]:
# Splitting age list by year
age_2012 = []
age_2013 = []
age_2014 = []
age_2015 = []
age_2016 = []

# For Loop

for num in range(95):
    
    if num <= 18:
        age_2012.append(age_list[num])
    elif num > 18 and num <= 37:
        age_2013.append(age_list[num])
    elif num > 37 and num <= 56:
        age_2014.append(age_list[num])
    elif num > 56 and num <= 75:
        age_2015.append(age_list[num])
    elif num > 75:
        age_2016.append(age_list[num])
    

In [7]:
# Splitting ethnicity list by year
ethnicity_2012 = []
ethnicity_2013 = []
ethnicity_2014 = []
ethnicity_2015 = []
ethnicity_2016 = []

# For Loop

for num in range(95):
    
    if num <= 18:
        ethnicity_2012.append(ethnicity_list[num])
    elif num > 18 and num <= 37:
        ethnicity_2013.append(ethnicity_list[num])
    elif num > 37 and num <= 56:
        ethnicity_2014.append(ethnicity_list[num])
    elif num > 56 and num <= 75:
        ethnicity_2015.append(ethnicity_list[num])
    elif num > 75:
        ethnicity_2016.append(ethnicity_list[num])

In [8]:
# Splitting age list by year
income_2012 = []
income_2013 = []
income_2014 = []
income_2015 = []
income_2016 = []

# For Loop

for num in range(95):
    
    if num <= 18:
        income_2012.append(income_list[num])
    elif num > 18 and num <= 37:
        income_2013.append(income_list[num])
    elif num > 37 and num <= 56:
        income_2014.append(income_list[num])
    elif num > 56 and num <= 75:
        income_2015.append(income_list[num])
    elif num > 75:
        income_2016.append(income_list[num])

In [9]:
# Splitting age list by year
edu_2012 = []
edu_2013 = []
edu_2014 = []
edu_2015 = []
edu_2016 = []

# For Loop

for num in range(95):
    
    if num <= 18:
        edu_2012.append(edu_list[num])
    elif num > 18 and num <= 37:
        edu_2013.append(edu_list[num])
    elif num > 37 and num <= 56:
        edu_2014.append(edu_list[num])
    elif num > 56 and num <= 75:
        edu_2015.append(edu_list[num])
    elif num > 75:
        edu_2016.append(edu_list[num])

In [16]:
# Use iterative loop to convert valus into integers & sum the values in the age list,
# thus combining male and female data

age_combine = []

for num in range(19):
    
    city = age_2016[num][0]
    pop = int(age_2016[num][1])
    tot1819 = int(age_2016[num][2]) + int(age_2016[num][10])
    tot20 = int(age_2016[num][3]) + int(age_2016[num][11])
    tot21 = int(age_2016[num][4]) + int(age_2016[num][12])
    tot2224 = int(age_2016[num][5]) + int(age_2016[num][13])
    tot2529 = int(age_2016[num][6]) + int(age_2016[num][14])
    tot3034 = int(age_2016[num][7]) + int(age_2016[num][15])
    tot3539 = int(age_2016[num][8]) + int(age_2016[num][16])
    tot4044 = int(age_2016[num][9]) + int(age_2016[num][17])
    
    tot1829 = tot1819 + tot20 + tot21 + tot2224 + tot2529
    tot3044 = tot3034 + tot3539 + tot4044
    
    
    age_combine.append([city,pop,tot1829,tot3044])
    
age_combine
    

[['Atlanta city, Georgia', 456378, 111563, 109761],
 ['Seattle city, Washington', 668849, 153546, 174781],
 ['Orlando city, Florida', 263306, 57282, 64564],
 ['New York city, New York', 8461961, 1598367, 1890517],
 ['Carson city, California', 92780, 16827, 17290],
 ['Newark city, New Jersey', 280139, 56080, 62773],
 ['Portland city, Oregon', 620589, 112933, 165930],
 ['Minneapolis city, Minnesota', 404670, 105223, 94989],
 ['San Jose city, California', 1009363, 171082, 228996],
 ['Kansas City city, Kansas', 149755, 25039, 30355],
 ['Foxborough CDP, Massachusetts', 5491, 615, 1010],
 ['Sandy city, Utah', 93141, 14245, 18487],
 ['Washington city, District of Columbia', 659009, 159451, 163879],
 ['Houston city, Texas', 2240582, 442457, 508738],
 ['Bridgeview village, Illinois', 16381, 2275, 3307],
 ['Chester city, Pennsylvania', 34056, 8459, 5488],
 ['Columbus city, Ohio', 837038, 195925, 186045],
 ['Commerce City city, Colorado', 51731, 7365, 12990],
 ['Frisco city, Texas', 145646, 13487

In [15]:
# Iterative loop to convert values into integers so that income levels may be added together to decrease
# the number of columns in the final DataFrame

income_combine = []

for num in range(19):
    
    city = income_2016[num][0]
    pop = int(income_2016[num][1])
    tot2 = int(income_2016[num][2])
    tot345 = int(income_2016[num][3]) + int(income_2016[num][4]) + int(income_2016[num][5])
    tot678 = int(income_2016[num][6]) + int(age_2016[num][7]) + int(income_2016[num][8])
    tot91011 = int(age_2016[num][9]) + int(age_2016[num][10]) + int(income_2016[num][11])
    tot121314 = int(age_2016[num][12]) + int(age_2016[num][13]) + int(income_2016[num][14])
    tot151617 = int(age_2016[num][15]) + int(age_2016[num][16]) + int(income_2016[num][17])
    tot181920 = int(age_2016[num][18]) + int(age_2016[num][19]) + int(income_2016[num][20])
    
    income_combine.append([city,pop,tot2,tot345,tot678,tot91011,tot121314,tot151617,tot181920])
    
income_combine

[['Atlanta city, Georgia', 81417, 679, 1887, 23387, 27694, 19713, 44007, 9635],
 ['Seattle city, Washington',
  140436,
  1194,
  243,
  36134,
  35844,
  25344,
  61720,
  72458],
 ['Orlando city, Florida', 38542, 306, 526, 12336, 12374, 10598, 28240, 56644],
 ['New York city, New York',
  1000242,
  12026,
  11196,
  353852,
  376033,
  262710,
  698431,
  96424],
 ['Carson city, California', 18979, 338, 246, 2993, 4315, 2992, 6318, 12969],
 ['Newark city, New Jersey',
  20502,
  539,
  251,
  11323,
  14683,
  8531,
  23785,
  54379],
 ['Portland city, Oregon',
  136114,
  1912,
  1547,
  31621,
  31691,
  17104,
  65989,
  76839],
 ['Minneapolis city, Minnesota',
  80520,
  736,
  573,
  20909,
  21375,
  18017,
  41967,
  51498],
 ['San Jose city, California',
  181122,
  1749,
  950,
  41729,
  50666,
  26322,
  75377,
  71740],
 ['Kansas City city, Kansas',
  30796,
  675,
  1476,
  7153,
  8649,
  6889,
  12262,
  36802],
 ['Foxborough CDP, Massachusetts', 990, 14, 0, 153, 235,

In [24]:
# Creating pandas DataFrames for each data list, starting with age

age_df = pd.DataFrame(age_combine)

# Rename columns for reference

age_df.columns=["City & State","Total Population","Age 18 to 29","Age 30 to 44"]

# Save DataFrame

age_df.to_csv("Output/Age_Data_2016.csv")

# Display the DataFrame

age_df


,City & State,Total Population,Age 18 to 29,Age 30 to 44
0,"Atlanta city, Georgia",456378,111563,109761
1,"Seattle city, Washington",668849,153546,174781
2,"Orlando city, Florida",263306,57282,64564
3,"New York city, New York",8461961,1598367,1890517
4,"Carson city, California",92780,16827,17290
5,"Newark city, New Jersey",280139,56080,62773
6,"Portland city, Oregon",620589,112933,165930
7,"Minneapolis city, Minnesota",404670,105223,94989
8,"San Jose city, California",1009363,171082,228996
9,"Kansas City city, Kansas",149755,25039,30355


In [ ]:
# Create new dataframe using previous dataframe to hold percentile values




In [20]:
# DataFrame for ethnicity

ethnicity_df = pd.DataFrame(ethnicity_2016)

# Rename columns

ethnicity_df.columns=["City & State","Total Population","White","Black/Afro-American","Latino","Asian","State Code","City Code"]

# Remove extraneous columns

ethnicity_df = ethnicity_df.loc[:, ["City & State","Total Population","White","Black/Afro-American","Latino","Asian"]]

# Save DataFrame

ethnicity_df.to_csv("Output/Ethnicity_Data_2016.csv")

# Display the DataFrame

ethnicity_df

,City & State,Total Population,White,Black/Afro-American,Latino,Asian
0,"Atlanta city, Georgia",456378,182794,239005,21838,18235
1,"Seattle city, Washington",668849,462673,47493,43868,94619
2,"Orlando city, Florida",263306,160607,69572,77009,10109
3,"New York city, New York",8461961,3644982,2064927,2457945,1159883
4,"Carson city, California",92780,27282,19830,37011,23963
5,"Newark city, New Jersey",280139,68226,140750,100950,4914
6,"Portland city, Oregon",620589,482410,35465,60040,47045
7,"Minneapolis city, Minnesota",404670,262130,76070,38832,23911
8,"San Jose city, California",1009363,425883,31096,329135,344188
9,"Kansas City city, Kansas",149755,88953,37530,43727,5786


In [21]:
# DataFrame for Income Level

income_df = pd.DataFrame(income_combine)

# Rename the columns

income_df.columns=["City & State","Total Households","Less than 10k","10k to 24k","25k to 39k","40k to 69k",\
                  "77k to 99k","100k to 174k","175k to 300k"]

# Save DataFrame

income_df.to_csv("Output/Income_Data_2016.csv")

# Display the DataFrame

income_df

,City & State,Total Households,Less than 10k,10k to 24k,25k to 39k,40k to 69k,77k to 99k,100k to 174k,175k to 300k
0,"Atlanta city, Georgia",81417,679,1887,23387,27694,19713,44007,9635
1,"Seattle city, Washington",140436,1194,243,36134,35844,25344,61720,72458
2,"Orlando city, Florida",38542,306,526,12336,12374,10598,28240,56644
3,"New York city, New York",1000242,12026,11196,353852,376033,262710,698431,96424
4,"Carson city, California",18979,338,246,2993,4315,2992,6318,12969
5,"Newark city, New Jersey",20502,539,251,11323,14683,8531,23785,54379
6,"Portland city, Oregon",136114,1912,1547,31621,31691,17104,65989,76839
7,"Minneapolis city, Minnesota",80520,736,573,20909,21375,18017,41967,51498
8,"San Jose city, California",181122,1749,950,41729,50666,26322,75377,71740
9,"Kansas City city, Kansas",30796,675,1476,7153,8649,6889,12262,36802


In [22]:
# DataFrame for Education Level

edu_df = pd.DataFrame(edu_2016)

# Rename the columns

edu_df.columns=["City & State","Edu Total Pop","Less Than High School Grad","High School Grad","Some College or Associates",\
               "Bachelor's Degree or Higher","State Code","City Code"]

# Remove extraneous columns

edu_df = edu_df.loc[:, ["City & State","Edu Total Pop","Less Than High School Grad","High School Grad","Some College or Associates",\
               "Bachelor's Degree or Higher"]]

# Save DataFrame

edu_df.to_csv("Output/Edu_Data_2016.csv")

# Display the dataframe

edu_df


,City & State,Edu Total Pop,Less Than High School Grad,High School Grad,Some College or Associates,Bachelor's Degree or Higher
0,"Atlanta city, Georgia",258046,23374,46860,57206,130606
1,"Seattle city, Washington",413133,21214,38922,93974,259023
2,"Orlando city, Florida",153051,13306,35951,47965,55829
3,"New York city, New York",4752442,778896,1092575,1037451,1843520
4,"Carson city, California",48771,8577,10814,16800,12580
5,"Newark city, New Jersey",152531,36047,55367,38992,22125
6,"Portland city, Oregon",381246,29657,56281,109969,185339
7,"Minneapolis city, Minnesota",232168,23886,36065,58364,113853
8,"San Jose city, California",564344,87769,94749,143961,237865
9,"Kansas City city, Kansas",76939,17076,24198,22884,12781
